In [31]:
import numpy as np
import pandas as pd

### Este notebook auna todo el preprocesado necesario para transformar los datos de la carpeta RAW a PREPARED:

In [32]:
path_in  = "../raw/trabajo6.csv"
path_out = "../partitioned/"

In [44]:
df = pd.read_csv(path_in)

### One Hot Encoding

In [34]:
df = df.join(pd.get_dummies(df['type']))

### Nuevas variables

In [35]:
df['diffOrigen'] = df['newbalanceOrig'] - df['oldbalanceOrg']
df['diffDestino'] = df['newbalanceDest'] - df['oldbalanceDest']

mean_oldOrg  = df['oldbalanceOrg'].mean()
mean_oldDest = df['oldbalanceDest'].mean()

# Para evitar NaN. No afecta la distribución de los datos.
df['cambioOrigen'] = (df['diffOrigen'] + 0.01)/(df['oldbalanceOrg'] + 1e3)
df['cambioDestino'] = (df['diffDestino'] + 0.01)/(df['oldbalanceDest'] + 1e3)

### Particionamos el dataset en 12 partes:

In [36]:
df['partition'] = np.random.randint(12, size=len(df))

In [37]:
df.partition.value_counts()

3     531487
1     531019
0     530594
8     530453
7     530403
2     530290
11    529943
6     529904
5     529809
4     529762
10    529683
9     529273
Name: partition, dtype: int64

### Write Prepared

In [38]:
# Necesita paquete 'pyarrow'

In [39]:
df.to_parquet(path_out, partition_cols='partition', compression='snappy')

In [24]:
# Para leer el parquet:
p_df = pd.read_parquet(path_out).drop(columns=['partition'])

In [41]:
df.type.unique().tolist()

['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN']

In [51]:
data = df[(df.type == 'TRANSFER') & (df.oldbalanceOrg != 0) & (df.newbalanceOrig != 0) & (df.oldbalanceDest != 0) & (df.newbalanceDest != 0)]
data_fraud = data[data.isFraud == 1]

In [52]:
data_fraud

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
60853,9,TRANSFER,994453.2,C1121789613,1437370.87,442917.67,C254839817,194812.76,665743.67,1,0


In [66]:
df[(df.type == 'CASH_OUT')].value_counts('isFraud')[1]

4116

In [67]:
dicc = {"juan":"es tonto"}

In [69]:
dicc['juan']

'es tonto'